# Chapter 18 - Gaussian Processes

Any model that is linear in its parameters with a Gaussian distribution over the parameters is a **Gaussian process**. This class spans discrete models, including random walks, and autoregressive processes, as well as continuous models, including Bayesian linear regression models, polynomials, Fourier series, radial basis functions, and even neural networks with an infinite number of hidden units.

## 18.1. Introduction to Gaussian Processes